In [2]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

# Disable eager execution (for TensorFlow v2 compatibility with v1 placeholders)
tf.compat.v1.disable_eager_execution()

# Load MNIST dataset
mnist = tfds.load('mnist', split=['train', 'test'], as_supervised=True)
train_data, test_data = mnist

# Function to preprocess data
def preprocess(dataset):
    images, labels = [], []
    for img, label in tfds.as_numpy(dataset):
        images.append(img.flatten() / 255.0)  # Normalize pixel values (0-1)
        labels.append(np.eye(10)[label])  # Convert to one-hot encoding
    return np.array(images), np.array(labels)

# Prepare train and test datasets
train_images, train_labels = preprocess(train_data)
test_images, test_labels = preprocess(test_data)

# Define network parameters
input_size = 784  # 28x28 pixels
hidden_size1 = 128
hidden_size2 = 64
output_size = 10  # 10 classes (digits 0-9)
learning_rate = 0.01
epochs = 20
batch_size = 100

# Define placeholders for input and output
X = tf.compat.v1.placeholder(tf.float32, [None, input_size])
Y = tf.compat.v1.placeholder(tf.float32, [None, output_size])

# Initialize weights and biases
weights = {
    'w1': tf.Variable(tf.random.normal([input_size, hidden_size1])),
    'w2': tf.Variable(tf.random.normal([hidden_size1, hidden_size2])),
    'w3': tf.Variable(tf.random.normal([hidden_size2, output_size]))
}

biases = {
    'b1': tf.Variable(tf.random.normal([hidden_size1])),
    'b2': tf.Variable(tf.random.normal([hidden_size2])),
    'b3': tf.Variable(tf.random.normal([output_size]))
}

# Define the feed-forward network
def neural_network(x):
    layer1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer1 = tf.nn.sigmoid(layer1)  # Using Sigmoid

    layer2 = tf.add(tf.matmul(layer1, weights['w2']), biases['b2'])
    layer2 = tf.nn.sigmoid(layer2)  # Using Sigmoid

    output_layer = tf.add(tf.matmul(layer2, weights['w3']), biases['b3'])
    return output_layer

# Forward propagation
logits = neural_network(X)
prediction = tf.nn.softmax(logits)

# Define loss function and optimizer (Backpropagation)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(loss)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) * 100

# Train the model
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())

    for epoch in range(epochs):
        total_batches = len(train_images) // batch_size
        avg_loss = 0

        for i in range(total_batches):
            batch_x = train_images[i * batch_size:(i + 1) * batch_size]
            batch_y = train_labels[i * batch_size:(i + 1) * batch_size]
            _, batch_loss = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y})
            avg_loss += batch_loss / total_batches

        acc = sess.run(accuracy, feed_dict={X: test_images, Y: test_labels})
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Test Accuracy: {acc:.2f}%")  # Show accuracy in %

    # Final test accuracy in percentage
    test_acc = sess.run(accuracy, feed_dict={X: test_images, Y: test_labels})
    print(f"\nFinal Test Accuracy: {test_acc:.2f}% ")


Epoch 1, Loss: 0.4912, Test Accuracy: 92.50%
Epoch 2, Loss: 0.2029, Test Accuracy: 94.13%
Epoch 3, Loss: 0.1489, Test Accuracy: 94.90%
Epoch 4, Loss: 0.1180, Test Accuracy: 95.38%
Epoch 5, Loss: 0.1066, Test Accuracy: 95.31%
Epoch 6, Loss: 0.0938, Test Accuracy: 95.41%
Epoch 7, Loss: 0.0846, Test Accuracy: 95.65%
Epoch 8, Loss: 0.0795, Test Accuracy: 96.11%
Epoch 9, Loss: 0.0734, Test Accuracy: 95.72%
Epoch 10, Loss: 0.0622, Test Accuracy: 96.26%
Epoch 11, Loss: 0.0611, Test Accuracy: 95.49%
Epoch 12, Loss: 0.0631, Test Accuracy: 96.11%
Epoch 13, Loss: 0.0604, Test Accuracy: 95.91%
Epoch 14, Loss: 0.0593, Test Accuracy: 95.52%
Epoch 15, Loss: 0.0612, Test Accuracy: 96.22%
Epoch 16, Loss: 0.0500, Test Accuracy: 96.23%
Epoch 17, Loss: 0.0467, Test Accuracy: 95.74%
Epoch 18, Loss: 0.0492, Test Accuracy: 96.65%
Epoch 19, Loss: 0.0441, Test Accuracy: 96.37%
Epoch 20, Loss: 0.0426, Test Accuracy: 96.52%

Final Test Accuracy: 96.52% 
